In [276]:
from flask import Flask, render_template_string, request, url_for, jsonify
import os

app = Flask(__name__)

# Initialize the neutrality score at 0.5
neutrality_score = 0.5
# create lists to store liked and disliked posts
liked_posts = set()
disliked_posts = set()
# List of available images in the static folder
folder_path = "static"

# Get a list of all file names in the folder
available_images = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]

#start page template
start_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Start Page</title>
    <link href="https://fonts.googleapis.com/css2?family=Baskervville&display=swap" rel="stylesheet">
    <style>
    body {
        display: grid;
        place-items: center;
        font-family: 'Baskervville', serif;
        background-color: #F5EFE0;
    
    }
    .start-button {
         margin-top: 30px;
            padding: 15px 30px;
            font-family: inherit;
            font-size: 30px;
            background-color: #E8DEC9;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            }
    @keyframes shrink {
        0% {transform: scale(1); }
        100% {transform: scale(0.8); }
        }
    .shrink-image {
        width: 600px;
        height: auto;
        animation: shrink 2s ease-in-out forwards;
        animation-delay: 0.3s;
    }
    </style>
</head>

<body>
    
    <img src="{{ url_for('static', filename='rabbitlogo-removebg.png') }}" alt="Logo" class="shrink-image" >
    <h2 style="text-align:center;"> Welcome to Rabbit, an interactive environment <br> where you can experience World War 1 brought into the modern day. <br>
    Click start to be brought into the social media simulation. </h2>
     <form action="/home" method="get">
        <button type="submit" class="start-button">Start</button>
    </form>
</body>
</html>

"""

# Main page HTML template with clickable images and image scores
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Flask Button Example</title>
     <style>
        body {
            background-color: #F5EFE0;
            padding-top: 10px;
        }

        .banner {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 80px; /* Adjust height as needed */
            background-color: #E8DEC9; /* A slightly darker shade for contrast */
            display: flex;
            padding: 10px 20px;
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
        }

    ]
        .banner img {
            height: 120%;
            width: auto;
            object-fit: contain;
        }
        
        .image-text-box {
            display: flex;        
            flex-direction: column; 
            text-overflow: ellipsis;
            justify-content: flex-end;
            text-align: center;
            height:530px;
            width:400px;
            background-color: #E8DEC9;
            padding: 25px;
            font-size: 24px;
            font-family: 'Baskervville', serif;
            margin-top: 100px; 
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
            border-radius: 15px;
        }

        .image-container {
            display: flex;
            justify-content: center; /* Centers horizontally */
            align-items: center; /* Aligns items in the middle */
            gap: 20px; /* Adds spacing between images */
            margin-top: 200px; /* Adds some space from the top */
        
        }
        .image-container img {
            width: 400px;
            height: auto;
            display: block;
            border-radius: 15px;
        }
        .image-wrapper {
            position:relative;
        }
    .like-btn {
        position: absolute;
        bottom: 30px; /* Position the button at the bottom */
        left: 0px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    .dislike-btn {
        position: absolute;
        bottom: 27px; /* Position the button at the bottom */
        left: 50px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    @keyframes buttonClick {
        0% {transform: scale(1);}
        50% {transform: scale(1.3);}
        100% {transform: scale(1);}
    }
   .like-btn.clicked, .dislike-btn.clicked {
    animation: buttonClick 0.3s ease-in-out;
}
</style>
        
<script>
function handleLike(postId) {
    fetch("/like", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const likeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='like']`);
            likeButton.classList.add("clicked");  // Add the "clicked" class
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);

            // Remove the "clicked" class after the animation ends
            likeButton.addEventListener("animationend", () => {
                likeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}

function handleDislike(postId) {
    fetch("/dislike", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const dislikeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='dislike']`);
            dislikeButton.classList.add("clicked");  // Add the "clicked" class
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);

            // Remove the "clicked" class after the animation ends
            dislikeButton.addEventListener("animationend", () => {
                dislikeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}
</script>
   
</head>

<body>
        
    <div class="banner">
        <img src="{{ url_for('static', filename='rabbitlogo-removebg.png') }}" alt="Logo" >
    </div>


    <div class="image-container">

        <div class="image-wrapper">
        <form method="POST" action="/button-clicked/{{ images[0].filename }}" style="display: inline;">
            <button type="submit" style="border: none; background: none; padding: 0;">
                <div class="image-text-box">
                <h2> Victory on the Dardanelles front!... </h2>
                <img src="{{ url_for('static', filename=images[0].filename) }}" style="border:3px solid black; height:400px; width:400px;" alt="Click to see {{ images[0].filename }}" >
                </div>
            </button>
        </form>
   
    
        <button class="like-btn" data-post-id="{{ images[0].filename }}" data-button-type="like" onclick="handleLike('{{ images[0].filename }}')">
            <img src="{{ url_for('static', filename='happyrabbit.png') }}" alt="Like" style="width: 100px; height: auto;">
         </button>
         <button class="dislike-btn" data-post-id="{{ images[0].filename }}" data-button-type="dislike" onclick="handleDislike('{{ images[0].filename }}')">
            <img src="{{ url_for('static', filename='sadrabbit.png') }}" alt="Dislike" style="width: 100px; height: auto;" >
        </button>
        </div>
        
        <div class="image-wrapper">
        <form method="POST" action="/button-clicked/{{ images[1].filename }}" style="display: inline;">
            <button type="submit" style="border: none; background: none; padding: 0;">
            <div class="image-text-box">
                <h3> Yesterday, our forces struck hard at the enemy positions in Sedd-ul-Bahr, delivering a crushing blow! </h3>
                <img src="{{ url_for('static', filename=images[1].filename) }}" style="border:3px solid black; height:400px; width:400px;" alt="Click to see {{ images[1].filename }}" >
            </div>
            </button>
        </form>
        <button class="like-btn" data-post-id="{{ images[1].filename }}" data-button-type="like" onclick="handleLike('{{ images[1].filename }}')">
            <img src="{{ url_for('static', filename='happyrabbit.png') }}" alt="Like" style="width: 100px; height: auto;" >
        </button>
        <button class="dislike-btn" data-post-id="{{ images[1].filename }}" data-button-type="dislike" onclick="handleDislike('{{ images[1].filename }}')">
            <img src="{{ url_for('static', filename='sadrabbit.png') }}" alt="Dislike" style="width: 100px; height: auto;" >
        </button>
        </div>

    </div>

    

</body>

</html>
"""

# Template to display the clicked image
button_clicked_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Button Clicked</title>
    <style>
        body {
            background-color: #F5EFE0;
            padding-top: 10px;
        }

        .banner {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 80px; /* Adjust height as needed */
            background-color: #E8DEC9; /* A slightly darker shade for contrast */
            display: flex;
            padding: 10px 20px;
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
        }

    ]
        .banner img {
            height: 120%;
            width: auto;
            object-fit: contain;
        }
        
        .image-text-box {
            display: flex;        
            flex-direction: column; 
            text-overflow: ellipsis;
            justify-content: flex-end;
            text-align: center;
            height:530px;
            width:400px;
            background-color: #E8DEC9;
            padding: 25px;
            font-size: 24px;
            font-family: 'Baskervville', serif;
            margin-top: 100px; 
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
            border-radius: 15px;
        }

        .image-container {
            display: flex;
            justify-content: center; /* Centers horizontally */
            align-items: center; /* Aligns items in the middle */
            gap: 20px; /* Adds spacing between images */
            margin-top: 200px; /* Adds some space from the top */
        
        }
        .image-container img {
            width: 400px;
            height: auto;
            display: block;
            border-radius: 15px;
        }
        .image-wrapper {
            position:relative;
        }
    .like-btn {
        position: absolute;
        bottom: 30px; /* Position the button at the bottom */
        left: 0px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    .dislike-btn {
        position: absolute;
        bottom: 27px; /* Position the button at the bottom */
        left: 50px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    @keyframes buttonClick {
        0% {transform: scale(1);}
        50% {transform: scale(1.3);}
        100% {transform: scale(1);}
    }
   .like-btn.clicked, .dislike-btn.clicked {
    animation: buttonClick 0.3s ease-in-out;
}
</style>
        
<script>
function handleLike(postId) {
    fetch("/like", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const likeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='like']`);
            likeButton.classList.add("clicked");  // Add the "clicked" class
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);

            // Remove the "clicked" class after the animation ends
            likeButton.addEventListener("animationend", () => {
                likeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}

function handleDislike(postId) {
    fetch("/dislike", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const dislikeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='dislike']`);
            dislikeButton.classList.add("clicked");  // Add the "clicked" class
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);

            // Remove the "clicked" class after the animation ends
            dislikeButton.addEventListener("animationend", () => {
                dislikeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}
</script>
   
</head>

<body>


    <div class="banner">
        <img src="{{ url_for('static', filename='rabbitlogo-removebg.png') }}" alt="Logo" >
    </div>
<div class="image-container">

    <div class="image-text-box">
    <img src="{{ url_for('static', filename=image_filename) }}" style="border:3px solid black; height:400px; width:400px;"alt="Clicked Image">
    </div>

    <div class="image-text-box">
    <p> Victory on the Dardanelles front! Our brave troops silenced the enemy near Sedd-ul-Bahr, inflicting devastating losses of over 15,000! The enemy cowers in defeat, leaving their fallen behind. Together, we march towards glory! #NationalPride #Victory</p>
    </div>

</div>
    <form action="/home" method="get">
        <button type="submit">Back to Home</button>
    </form>
</body>
</html>
"""


@app.route("/", methods=["GET"])
def start():
        return render_template_string(start_template)

@app.route("/home", methods=["GET"])
def home():
    global neutrality_score
    # Randomly select two different images for display
    nr_images = len(available_images)
    
    # Calculate image score based on position
    images = []
    for idx, filename in enumerate(available_images):
        image_score = idx / nr_images  # Image score calculation
        images.append({'filename': filename, 'score': round(image_score, 2)})  # Store both filename and score
    
    # Select two images based on the neutrality_score
    
    if neutrality_score == 1:
        image1 = int(neutrality_score * nr_images)-1
        image2 = image1 - 1
    elif neutrality_score == 0:
        image1 = int(neutrality_score * nr_images)
        image2 = image1 + 1
    else:
        image1 = int(neutrality_score * nr_images)
        image2 = image1 + 1

    selected_images = [images[image1], images[image2]]
    return render_template_string(html_template, neutrality_score=neutrality_score, images=selected_images)

@app.route("/button-clicked/<image_filename>", methods=["POST"])
def button_clicked(image_filename):
    global neutrality_score
    
    # Recalculate the image list (just like in the home route) to get the image score
    nr_images = len(available_images)
    images = []
    for idx, filename in enumerate(available_images):
        image_score = idx / nr_images  # Image score calculation
        images.append({'filename': filename, 'score': round(image_score, 2)})  # Store both filename and score
    
    # Find the image with the corresponding filename
    image = next(image for image in images if image['filename'] == image_filename)
    image_score = image['score']  # Get the image score
    
    return render_template_string(button_clicked_template, neutrality_score=neutrality_score, image_filename=image_filename, image_score=image_score)

@app.route("/like", methods=["POST"])
def like():
    global neutrality_score
    post_id = request.json.get("post_id")
    liked_posts.add(post_id)
    neutrality_score = min(1.0, neutrality_score + 0.1)
    return jsonify({"success": True, "neutrality_score": neutrality_score})


@app.route("/dislike", methods=["POST"])
def dislike():
    global neutrality_score
    post_id = request.json.get("post_id")
    disliked_posts.add(post_id)
    neutrality_score = max(0.0, neutrality_score - 0.1)

    return jsonify({"success": True, "neutrality_score": neutrality_score})
# Run the Flask app
from werkzeug.serving import run_simple

if __name__ == "__main__":
    run_simple("localhost", 5001, app)

 * Running on http://localhost:5001
Press CTRL+C to quit
[2025-02-16 23:27:10,966] ERROR in app: Exception on /button-clicked/happyrabbit.png [POST]
Traceback (most recent call last):
  File "/Users/keziahdutt/Downloads/Datathon 2025/rabbitenv/lib/python3.10/site-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/keziahdutt/Downloads/Datathon 2025/rabbitenv/lib/python3.10/site-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/keziahdutt/Downloads/Datathon 2025/rabbitenv/lib/python3.10/site-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/keziahdutt/Downloads/Datathon 2025/rabbitenv/lib/python3.10/site-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "/var/folders/c5/7b2hdd417y90r2mfyk9z39mw0000g